In [2]:
import os
import getpass
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.cohere import CohereEmbedding
from llama_parse import LlamaParse
from llama_index.core.node_parser import MarkdownNodeParser, SentenceSplitter
from llama_index.core import VectorStoreIndex, QueryBundle, Response, Document, Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.elasticsearch import ElasticsearchStore
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.embeddings import resolve_embed_model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from dotenv import load_dotenv
import cohere
import time


import nest_asyncio

nest_asyncio.apply()

In [3]:
load_dotenv()

api_keys = os.getenv("LLAMA_INDEX_KEYS").split(',')
for i, api_key in enumerate(api_keys[1:]):
    api_keys[i+1] = api_key[1:]

ELASTIC_CLOUD_ID = os.getenv("ELASTIC_CLOUD_ID_MISTRAL")
ELASTIC_API_KEY = os.getenv("ELASTIC_API_KEY_MISTRAL")

In [4]:
es_vector_store = ElasticsearchStore(index_name="calls",
                                     vector_field='conversation_vector',
                                     text_field='conversation',
                                     es_cloud_id=ELASTIC_CLOUD_ID,
                                     es_api_key=ELASTIC_API_KEY)

In [5]:
# Local LLM to send user query to
Settings.llm = Ollama(model="llama3:instruct", request_timeout=60.0)
Settings.embed_model= resolve_embed_model("local:BAAI/bge-large-en-v1.5")

index = VectorStoreIndex.from_vector_store(es_vector_store)
query_engine = index.as_query_engine(Settings.llm, similarity_top_k=10)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [6]:
questions = []
with open('Questions.txt', 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()  # Remove any leading/trailing whitespace
        if line.endswith('?'):  # Check if the line ends with a question mark
            questions.append(line)

In [7]:
query_results = []
i = 0
for query in questions:
    bundle = QueryBundle(query, embedding=Settings.embed_model.get_query_embedding(query))
    result = query_engine.query(bundle)
    query_results.append({
        "query": query,
        "result": result.response
    })
    print(i)
    i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [8]:
query_results

[{'query': 'What is the fiscal year end date for Apple Inc. as reported in their 2022 Form 10-K?',
  'result': "Based on the provided context information from Apple Inc.'s 2022 Form 10-K, the fiscal year end date is September 30th."},
 {'query': 'How many shares of common stock were outstanding for Apple Inc. as of October 14, 2022?',
  'result': 'Based on the provided context information, we can see that as of September 29, 2018, there were 4,754,986 shares of common stock issued and outstanding. As of September 26, 2020, there were 16,426,786 shares of common stock issued and outstanding.\n\nHowever, we are not provided with the number of shares outstanding for Apple Inc. as of October 14, 2022. The context information only mentions that as of October 14, 2022, there were 23,838 shareholders of record, but it does not provide the number of shares outstanding.\n\nTherefore, I cannot answer the query without prior knowledge or additional context information.'},
 {'query': "What is the 

In [10]:
import json

json_data = json.dumps(query_results)

with open('query_results.json', 'w') as file:
    file.write(json_data)